In [1]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Wedge
import io
import base64

# --- Physical Constants ---
G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)
C = 299792458    # Speed of light (m/s)
SOLAR_MASS = 1.989e30  # Solar mass (kg)
AU = 1.496e11    # Astronomical Unit (m)
PLANCK_LENGTH = 1.616255e-35  # Planck length (m)
PARSEC = 3.086e16  # Parsec (m)

class RelativisticCalculator:
    def __init__(self, mass_multiplier, distance_offset_log, spin_param=0, observer_velocity=0):
        self.M = mass_multiplier * 1e6 * SOLAR_MASS
        self.a = spin_param  # Dimensionless spin parameter (0 to 0.998)
        self.v_obs = observer_velocity * C  # Observer velocity as fraction of c
        
        # Schwarzschild radius
        self.Rs = (2 * G * self.M) / (C**2)
        
        # Kerr metric parameters
        self.r_g = G * self.M / (C**2)  # Gravitational radius
        self.r_isco = self.calculate_isco()  # Innermost stable circular orbit
        self.r_photon = self.calculate_photon_sphere()
        
        # Distance from center
        self.r = self.Rs + (10**distance_offset_log)
        
        # Calculate various effects
        self.gravitational_dilation = self.calc_gravitational_dilation()
        self.frame_dragging = self.calc_frame_dragging()
        self.doppler_shift = self.calc_doppler_effect()
        self.total_dilation = self.calc_total_dilation()
        self.tidal_force = self.calc_tidal_forces()
        self.escape_velocity = self.calc_escape_velocity()
        self.orbital_velocity = self.calc_orbital_velocity()
        self.hawking_temp = self.calc_hawking_temperature()
        self.gravitational_redshift = self.calc_gravitational_redshift()
        self.geodesic_precession = self.calc_geodesic_precession()
        
    def calculate_isco(self):
        """Calculate innermost stable circular orbit for Kerr black hole"""
        a = self.a
        Z1 = 1 + (1 - a**2)**(1/3) * ((1 + a)**(1/3) + (1 - a)**(1/3))
        Z2 = np.sqrt(3 * a**2 + Z1**2)
        r_isco = self.r_g * (3 + Z2 - np.sqrt((3 - Z1) * (3 + Z1 + 2*Z2)))
        return r_isco
    
    def calculate_photon_sphere(self):
        """Calculate photon sphere radius"""
        return 1.5 * self.Rs * (1 + np.cos(2/3 * np.arccos(-self.a)))
    
    def calc_gravitational_dilation(self):
        """Schwarzschild time dilation factor"""
        return np.sqrt(1 - (self.Rs / self.r))
    
    def calc_frame_dragging(self):
        """Frame dragging angular velocity (Kerr metric)"""
        r = self.r
        a = self.a * self.r_g
        omega = (2 * a * G * self.M) / (C * r**3)
        return omega
    
    def calc_doppler_effect(self):
        """Special relativistic Doppler effect"""
        gamma = 1 / np.sqrt(1 - (self.v_obs / C)**2)
        return 1 / gamma
    
    def calc_total_dilation(self):
        """Combined gravitational and kinematic time dilation"""
        return self.gravitational_dilation * self.doppler_shift
    
    def calc_tidal_forces(self):
        """Tidal acceleration gradient (m/s^2 per meter)"""
        return (2 * G * self.M) / (self.r**3)
    
    def calc_escape_velocity(self):
        """Escape velocity at distance r"""
        v_esc = np.sqrt(2 * G * self.M / self.r)
        return v_esc / C  # As fraction of c
    
    def calc_orbital_velocity(self):
        """Circular orbital velocity at distance r"""
        v_orb = np.sqrt(G * self.M / self.r)
        return v_orb / C  # As fraction of c
    
    def calc_hawking_temperature(self):
        """Hawking temperature of the black hole (K)"""
        hbar = 1.054571817e-34  # Reduced Planck constant
        k_B = 1.380649e-23  # Boltzmann constant
        T_H = (hbar * C**3) / (8 * np.pi * k_B * G * self.M)
        return T_H
    
    def calc_gravitational_redshift(self):
        """Gravitational redshift factor z"""
        return 1 / self.gravitational_dilation - 1
    
    def calc_geodesic_precession(self):
        """Geodesic precession rate (radians per orbit)"""
        return 6 * np.pi * G * self.M / (C**2 * self.r)
    
    def calc_energy_extraction_efficiency(self):
        """Penrose process maximum energy extraction efficiency"""
        if self.a == 0:
            return 0.0
        eta = 1 - np.sqrt(1 - self.a**2)
        return eta * 100  # As percentage

def create_visualization(calc):
    """Create detailed visualization of spacetime curvature"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 14))
    fig.patch.set_facecolor('#0e1117')
    
    # --- Plot 1: Schwarzschild Geometry ---
    ax1.set_facecolor('#1a1c24')
    theta = np.linspace(0, 2*np.pi, 1000)
    
    # Event horizon
    eh_x = calc.Rs/1e9 * np.cos(theta)
    eh_y = calc.Rs/1e9 * np.sin(theta)
    ax1.fill(eh_x, eh_y, color='black', alpha=1, label='Event Horizon')
    ax1.plot(eh_x, eh_y, color='#ff3366', linewidth=3)
    
    # Photon sphere
    ps_x = calc.r_photon/1e9 * np.cos(theta)
    ps_y = calc.r_photon/1e9 * np.sin(theta)
    ax1.plot(ps_x, ps_y, '--', color='#ffcc00', linewidth=2, label='Photon Sphere', alpha=0.7)
    
    # ISCO
    isco_x = calc.r_isco/1e9 * np.cos(theta)
    isco_y = calc.r_isco/1e9 * np.sin(theta)
    ax1.plot(isco_x, isco_y, '--', color='#00ffcc', linewidth=2, label='ISCO', alpha=0.7)
    
    # Current position
    pos_angle = np.pi/4
    pos_x = calc.r/1e9 * np.cos(pos_angle)
    pos_y = calc.r/1e9 * np.sin(pos_angle)
    ax1.plot(pos_x, pos_y, 'o', color='#00ff00', markersize=15, label='Observer Position')
    
    # Accretion disk
    for i in range(5):
        radius = calc.r_isco/1e9 + i * calc.Rs/1e9 * 0.5
        circle = Circle((0, 0), radius, fill=False, color='#ff9933', 
                       alpha=0.3-i*0.05, linewidth=1.5)
        ax1.add_patch(circle)
    
    ax1.set_xlim(-calc.r/1e9*1.5, calc.r/1e9*1.5)
    ax1.set_ylim(-calc.r/1e9*1.5, calc.r/1e9*1.5)
    ax1.set_xlabel('Distance (million km)', color='white', fontsize=11)
    ax1.set_ylabel('Distance (million km)', color='white', fontsize=11)
    ax1.set_title('Schwarzschild Geometry & Critical Radii', color='#00ffcc', 
                 fontsize=13, fontweight='bold')
    ax1.tick_params(colors='white')
    ax1.legend(loc='upper right', facecolor='#1a1c24', edgecolor='#00ffcc', 
              labelcolor='white', fontsize=9)
    ax1.grid(True, alpha=0.2, color='#444')
    ax1.set_aspect('equal')
    
    # --- Plot 2: Time Dilation vs Distance ---
    ax2.set_facecolor('#1a1c24')
    r_range = np.linspace(calc.Rs * 1.001, calc.Rs * 10, 1000)
    dilation_curve = np.sqrt(1 - calc.Rs / r_range)
    
    ax2.plot(r_range/calc.Rs, 1/dilation_curve, color='#00ffcc', linewidth=3)
    ax2.axvline(calc.r/calc.Rs, color='#ff3366', linestyle='--', linewidth=2, 
               label=f'Current Position (r={calc.r/calc.Rs:.4f} Rs)')
    ax2.axhline(1/calc.gravitational_dilation, color='#ff3366', linestyle='--', 
               linewidth=2, alpha=0.5)
    
    ax2.set_xlabel('Distance (Schwarzschild Radii)', color='white', fontsize=11)
    ax2.set_ylabel('Time Dilation Factor', color='white', fontsize=11)
    ax2.set_title('Gravitational Time Dilation Profile', color='#00ffcc', 
                 fontsize=13, fontweight='bold')
    ax2.set_xlim(1, 10)
    ax2.tick_params(colors='white')
    ax2.legend(facecolor='#1a1c24', edgecolor='#00ffcc', labelcolor='white', fontsize=9)
    ax2.grid(True, alpha=0.2, color='#444')
    ax2.set_yscale('log')
    
    # --- Plot 3: Tidal Forces ---
    ax3.set_facecolor('#1a1c24')
    r_range_tidal = np.linspace(calc.Rs * 1.001, calc.Rs * 50, 1000)
    tidal_forces = (2 * G * calc.M) / (r_range_tidal**3)
    
    ax3.plot(r_range_tidal/calc.Rs, tidal_forces, color='#ff3366', linewidth=3)
    ax3.axvline(calc.r/calc.Rs, color='#00ffcc', linestyle='--', linewidth=2, 
               label=f'Current: {calc.tidal_force:.2e} m/s²/m')
    
    ax3.set_xlabel('Distance (Schwarzschild Radii)', color='white', fontsize=11)
    ax3.set_ylabel('Tidal Gradient (m/s²/m)', color='white', fontsize=11)
    ax3.set_title('Tidal Force Gradient (Spaghettification)', color='#00ffcc', 
                 fontsize=13, fontweight='bold')
    ax3.tick_params(colors='white')
    ax3.legend(facecolor='#1a1c24', edgecolor='#00ffcc', labelcolor='white', fontsize=9)
    ax3.grid(True, alpha=0.2, color='#444')
    ax3.set_yscale('log')
    
    # --- Plot 4: Effective Potential ---
    ax4.set_facecolor('#1a1c24')
    r_pot = np.linspace(calc.Rs * 1.5, calc.Rs * 20, 1000)
    
    # Effective potential for different angular momenta
    L_values = [3, 3.5, 4, 5]  # Dimensionless angular momentum
    colors = ['#ff3366', '#ffcc00', '#00ffcc', '#9933ff']
    
    for L, color in zip(L_values, colors):
        V_eff = -1/r_pot + L**2/(2*r_pot**2) - (calc.Rs * L**2)/(r_pot**3)
        ax4.plot(r_pot/calc.Rs, V_eff * 1e6, color=color, linewidth=2.5, 
                label=f'L = {L} (GM/c)', alpha=0.8)
    
    ax4.axvline(calc.r/calc.Rs, color='white', linestyle='--', linewidth=2, alpha=0.5)
    ax4.set_xlabel('Distance (Schwarzschild Radii)', color='white', fontsize=11)
    ax4.set_ylabel('Effective Potential (×10⁻⁶)', color='white', fontsize=11)
    ax4.set_title('Effective Potential for Orbital Motion', color='#00ffcc', 
                 fontsize=13, fontweight='bold')
    ax4.tick_params(colors='white')
    ax4.legend(facecolor='#1a1c24', edgecolor='#00ffcc', labelcolor='white', fontsize=9)
    ax4.grid(True, alpha=0.2, color='#444')
    ax4.set_ylim(-3, 1)
    
    plt.tight_layout()
    
    # Convert plot to base64 for HTML embedding
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=100, facecolor='#0e1117')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode()
    plt.close()
    
    return img_base64

def format_time_elapsed(seconds):
    """Convert seconds to human-readable format"""
    years = seconds / (365.25 * 24 * 3600)
    
    if years >= 1e9:
        return f"{years/1e9:.3f} Billion Years"
    elif years >= 1e6:
        return f"{years/1e6:.3f} Million Years"
    elif years >= 1000:
        return f"{years/1000:.3f} Thousand Years"
    elif years >= 1:
        days = (years % 1) * 365.25
        hours = (days % 1) * 24
        return f"{int(years)} Years, {int(days)} Days, {int(hours)} Hours"
    else:
        days = years * 365.25
        hours = (days % 1) * 24
        minutes = (hours % 1) * 60
        return f"{int(days)} Days, {int(hours)} Hours, {int(minutes)} Minutes"

def update_dashboard(mass_m, dist_log, spin, velocity):
    clear_output(wait=True)
    
    calc = RelativisticCalculator(mass_m, dist_log, spin, velocity)
    
    # Time conversions for 1 hour on planet
    hour_on_planet = 3600  # seconds
    earth_seconds = hour_on_planet / calc.total_dilation
    earth_time_str = format_time_elapsed(earth_seconds)
    
    # Energy extraction efficiency
    penrose_efficiency = calc.calc_energy_extraction_efficiency()
    
    # Generate visualization
    img_b64 = create_visualization(calc)
    
    # Advanced metrics
    spaghettification_factor = calc.tidal_force * 2  # 2m human height
    orbital_period = 2 * np.pi * calc.r / (calc.orbital_velocity * C)
    coordinate_time_diff = earth_seconds - hour_on_planet
    
    # HTML Dashboard
    html_template = f"""
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&display=swap');
        .main-container {{
            background: linear-gradient(135deg, #0e1117 0%, #1a1c24 100%);
            border: 3px solid #00ffcc;
            border-radius: 15px;
            padding: 25px;
            font-family: 'Orbitron', 'Courier New', monospace;
            color: #00ffcc;
            box-shadow: 0 0 30px rgba(0, 255, 204, 0.3);
        }}
        .header {{
            text-align: center;
            color: #ff3366;
            font-size: 2.5em;
            font-weight: 900;
            text-shadow: 0 0 20px rgba(255, 51, 102, 0.8);
            margin-bottom: 20px;
            letter-spacing: 3px;
        }}
        .section {{
            background: #1a1c24;
            border-left: 5px solid #ff3366;
            padding: 20px;
            margin: 15px 0;
            border-radius: 8px;
            box-shadow: inset 0 0 20px rgba(0, 0, 0, 0.5);
        }}
        .section-title {{
            color: #ffcc00;
            font-size: 1.4em;
            font-weight: 700;
            margin-bottom: 15px;
            text-transform: uppercase;
            letter-spacing: 2px;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 10px 0;
        }}
        td {{
            padding: 12px;
            border-bottom: 1px solid #333;
            font-size: 1.05em;
        }}
        .label {{
            color: #aaa;
            font-weight: 600;
            width: 50%;
        }}
        .value {{
            color: #00ffcc;
            font-weight: 700;
            text-align: right;
        }}
        .critical {{
            color: #ff3366;
            font-weight: 900;
            font-size: 1.1em;
        }}
        .highlight {{
            background: linear-gradient(90deg, #ff3366 0%, #9933ff 100%);
            padding: 25px;
            border-radius: 10px;
            margin: 20px 0;
            text-align: center;
            box-shadow: 0 0 25px rgba(255, 51, 102, 0.5);
        }}
        .highlight-value {{
            font-size: 2.8em;
            color: #ffffff;
            font-weight: 900;
            text-shadow: 0 0 15px rgba(255, 255, 255, 0.8);
            margin: 15px 0;
        }}
        .warning {{
            background: #2a1a1a;
            border: 2px solid #ff3366;
            padding: 15px;
            border-radius: 8px;
            color: #ff6666;
            margin: 15px 0;
            font-weight: 600;
        }}
        .grid {{
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 15px;
            margin: 15px 0;
        }}
        .metric-box {{
            background: #252830;
            padding: 15px;
            border-radius: 8px;
            border: 2px solid #444;
            transition: all 0.3s;
        }}
        .metric-box:hover {{
            border-color: #00ffcc;
            box-shadow: 0 0 15px rgba(0, 255, 204, 0.3);
        }}
        .metric-label {{
            color: #888;
            font-size: 0.9em;
            margin-bottom: 8px;
        }}
        .metric-value {{
            color: #00ffcc;
            font-size: 1.4em;
            font-weight: 700;
        }}
        img {{
            width: 100%;
            border-radius: 10px;
            margin: 20px 0;
            border: 2px solid #00ffcc;
            box-shadow: 0 0 20px rgba(0, 255, 204, 0.2);
        }}
    </style>
    
    <div class="main-container">
        <div class="header">
            ⚛️ RELATIVISTIC SPACETIME ANALYZER ⚛️
        </div>
        
        <div class="section">
            <div class="section-title">🌌 Black Hole Parameters</div>
            <table>
                <tr>
                    <td class="label">Black Hole Mass:</td>
                    <td class="value">{mass_m:.2f} × 10⁶ M☉</td>
                </tr>
                <tr>
                    <td class="label">Total Mass (kg):</td>
                    <td class="value">{calc.M:.3e} kg</td>
                </tr>
                <tr>
                    <td class="label">Dimensionless Spin (a/M):</td>
                    <td class="value">{spin:.3f}</td>
                </tr>
                <tr>
                    <td class="label">Schwarzschild Radius:</td>
                    <td class="value critical">{calc.Rs/1e6:.4f} million km</td>
                </tr>
                <tr>
                    <td class="label">ISCO Radius:</td>
                    <td class="value">{calc.r_isco/1e6:.4f} million km ({calc.r_isco/calc.Rs:.3f} Rs)</td>
                </tr>
                <tr>
                    <td class="label">Photon Sphere:</td>
                    <td class="value">{calc.r_photon/1e6:.4f} million km ({calc.r_photon/calc.Rs:.3f} Rs)</td>
                </tr>
                <tr>
                    <td class="label">Hawking Temperature:</td>
                    <td class="value">{calc.hawking_temp:.3e} K</td>
                </tr>
            </table>
        </div>
        
        <div class="section">
            <div class="section-title">📍 Observer Position & Geometry</div>
            <table>
                <tr>
                    <td class="label">Distance from Singularity (r):</td>
                    <td class="value">{calc.r/1e6:.10f} million km</td>
                </tr>
                <tr>
                    <td class="label">Distance in Schwarzschild Radii:</td>
                    <td class="value critical">{calc.r/calc.Rs:.6f} Rs</td>
                </tr>
                <tr>
                    <td class="label">Proximity to Event Horizon:</td>
                    <td class="value">{(calc.r - calc.Rs):,.3f} meters</td>
                </tr>
                <tr>
                    <td class="label">Proximity in Planck Lengths:</td>
                    <td class="value">{(calc.r - calc.Rs)/PLANCK_LENGTH:.3e} ℓₚ</td>
                </tr>
                <tr>
                    <td class="label">Observer Velocity:</td>
                    <td class="value">{velocity:.3f} c ({velocity*C:,.0f} m/s)</td>
                </tr>
            </table>
        </div>
        
        <div class="highlight">
            <div style="font-size: 1.3em; color: #ffcc00; font-weight: 700;">
                ⏱️ TEMPORAL DISPLACEMENT CALCULATION
            </div>
            <div style="margin: 20px 0; color: white; font-size: 1.2em;">
                For every <span style="color: #00ffcc; font-weight: 900;">1 HOUR</span> experienced by the observer:
            </div>
            <div class="highlight-value">
                {earth_time_str}
            </div>
            <div style="color: #ffcc00; font-size: 1.1em; margin-top: 10px;">
                passes on Earth
            </div>
        </div>
        
        <div class="section">
            <div class="section-title">⚡ Relativistic Effects Analysis</div>
            
            <div class="grid">
                <div class="metric-box">
                    <div class="metric-label">Gravitational Time Dilation</div>
                    <div class="metric-value">{1/calc.gravitational_dilation:,.2f}×</div>
                </div>
                <div class="metric-box">
                    <div class="metric-label">Kinematic Dilation (SR)</div>
                    <div class="metric-value">{1/calc.doppler_shift:,.4f}×</div>
                </div>
                <div class="metric-box">
                    <div class="metric-label">Combined Dilation Factor</div>
                    <div class="metric-value critical">{1/calc.total_dilation:,.2f}×</div>
                </div>
                <div class="metric-box">
                    <div class="metric-label">Gravitational Redshift (z)</div>
                    <div class="metric-value">{calc.gravitational_redshift:.6f}</div>
                </div>
                <div class="metric-box">
                    <div class="metric-label">Frame Dragging (ω)</div>
                    <div class="metric-value">{calc.frame_dragging:.3e} rad/s</div>
                </div>
                <div class="metric-box">
                    <div class="metric-label">Geodesic Precession</div>
                    <div class="metric-value">{np.degrees(calc.geodesic_precession):.6f}° /orbit</div>
                </div>
            </div>
        </div>
        
        <div class="section">
            <div class="section-title">🚀 Orbital Mechanics</div>
            <table>
                <tr>
                    <td class="label">Escape Velocity:</td>
                    <td class="value critical">{calc.escape_velocity:.6f} c ({calc.escape_velocity*C:,.0f} m/s)</td>
                </tr>
                <tr>
                    <td class="label">Circular Orbital Velocity:</td>
                    <td class="value">{calc.orbital_velocity:.6f} c ({calc.orbital_velocity*C:,.0f} m/s)</td>
                </tr>
                <tr>
                    <td class="label">Orbital Period:</td>
                    <td class="value">{orbital_period/3600:.3f} hours ({orbital_period:.2f} s)</td>
                </tr>
                <tr>
                    <td class="label">Orbital Angular Momentum (specific):</td>
                    <td class="value">{calc.orbital_velocity * calc.r:.3e} m²/s</td>
                </tr>
            </table>
        </div>
        
        <div class="section">
            <div class="section-title">💀 Tidal Forces & Structural Stress</div>
            <table>
                <tr>
                    <td class="label">Tidal Gradient:</td>
                    <td class="value critical">{calc.tidal_force:.3e} m/s²/m</td>
                </tr>
                <tr>
                    <td class="label">Spaghettification Force (2m object):</td>
                    <td class="value critical">{spaghettification_factor:.3e} m/s²</td>
                </tr>
                <tr>
                    <td class="label">Tidal Acceleration (g-forces):</td>
                    <td class="value">{spaghettification_factor/9.81:,.0f} g</td>
                </tr>
            </table>
            
            {f'<div class="warning">⚠️ EXTREME DANGER: Tidal forces exceed survivable limits! Structural integrity compromised.</div>' if spaghettification_factor > 100 else ''}
        </div>
        
        <div class="section">
            <div class="section-title">🔬 Advanced Metrics</div>
            <table>
                <tr>
                    <td class="label">Coordinate Time Differential:</td>
                    <td class="value">{coordinate_time_diff/86400:.3f} days ({coordinate_time_diff:,.0f} s)</td>
                </tr>
                <tr>
                    <td class="label">Proper Time Elapsed (Observer):</td>
                    <td class="value">1.000 hour (3600.000 s)</td>
                </tr>
                <tr>
                    <td class="label">Coordinate Time Elapsed (Earth):</td>
                    <td class="value">{earth_seconds/3600:.2f} hours ({earth_seconds:,.0f} s)</td>
                </tr>
                <tr>
                    <td class="label">Kretschmann Scalar (curvature):</td>
                    <td class="value">{48 * (G * calc.M / C**2)**2 / calc.r**6:.3e} m⁻⁴</td>
                </tr>
                {f'<tr><td class="label">Penrose Process Efficiency:</td><td class="value">{penrose_efficiency:.2f}%</td></tr>' if spin > 0 else ''}
            </table>
        </div>
        
        <div class="section">
            <div class="section-title">📊 Spacetime Visualization</div>
            <img src="data:image/png;base64,{img_b64}" alt="Spacetime Visualization">
        </div>
        
        <div style="text-align: center; margin-top: 30px; color: #666; font-size: 0.9em;">
            <i>Calculations based on General Relativity (Einstein Field Equations) & Kerr Metric</i><br>
            <i>Units: G = {G:.3e} m³kg⁻¹s⁻², c = {C:,} m/s</i>
        </div>
    </div>
    """
    
    display(HTML(html_template))

# --- Interactive UI ---
print("🌌 RELATIVISTIC SPACETIME ANALYZER 🌌\n")

mass_slider = widgets.FloatLogSlider(
    value=100,
    base=10,
    min=0,
    max=4,
    step=0.01,
    description='BH Mass:',
    style={'description_width': '120px'},
    readout_format='.2f'
)

dist_slider = widgets.FloatSlider(
    value=3,
    min=-5,
    max=10,
    step=0.01,
    description='Distance Log:',
    style={'description_width': '120px'},
    readout_format='.3f'
)

spin_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.998,
    step=0.001,
    description='Spin (a/M):',
    style={'description_width': '120px'},
    readout_format='.3f'
)

velocity_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.99,
    step=0.001,
    description='Velocity (c):',
    style={'description_width': '120px'},
    readout_format='.3f'
)

# Create accordion for advanced controls
accordion = widgets.Accordion(children=[
    widgets.VBox([
        widgets.HTML("<b>Spin Parameter (Kerr Black Hole)</b>"),
        widgets.HTML("<i>0 = Schwarzschild (non-rotating), 0.998 = near-extremal Kerr</i>"),
        spin_slider
    ]),
    widgets.VBox([
        widgets.HTML("<b>Observer Velocity (Special Relativity)</b>"),
        widgets.HTML("<i>Velocity as fraction of speed of light</i>"),
        velocity_slider
    ])
])

accordion.set_title(0, '🔄 Rotation & Kerr Metric')
accordion.set_title(1, '🚀 Kinematic Effects')
accordion.selected_index = None

# Main UI container
ui = widgets.VBox([
    widgets.HTML("""
        <div style='background: linear-gradient(135deg, #1a1c24 0%, #0e1117 100%); 
                    padding: 20px; 
                    border-radius: 10px; 
                    border: 2px solid #00ffcc;
                    margin-bottom: 20px;'>
            <h2 style='color: #ff3366; text-align: center; margin: 0;'>
                ⚛️ RELATIVISTIC SPACETIME ANALYZER ⚛️
            </h2>
            <p style='color: #00ffcc; text-align: center; margin: 10px 0 0 0; font-size: 0.9em;'>
                Explore extreme gravitational time dilation near black holes
            </p>
        </div>
    """),
    widgets.HTML("<b style='color: #ffcc00;'>Primary Controls:</b>"),
    widgets.HBox([
        widgets.VBox([
            widgets.HTML("<b>Black Hole Mass (×10⁶ M☉):</b>"),
            mass_slider
        ], layout=widgets.Layout(width='50%')),
        widgets.VBox([
            widgets.HTML("<b>Distance Offset (log scale):</b>"),
            dist_slider
        ], layout=widgets.Layout(width='50%'))
    ]),
    widgets.HTML("<br><b style='color: #ffcc00;'>Advanced Parameters:</b>"),
    accordion,
    widgets.HTML("""
        <div style='margin-top: 15px; padding: 10px; background: #1a1c24; 
                    border-left: 3px solid #ff3366; border-radius: 5px;'>
            <small style='color: #aaa;'>
                <b>Tips:</b> Move sliders to explore different scenarios. 
                Lower distance offset = closer to event horizon = extreme time dilation.
                Increase spin for Kerr black hole effects (frame dragging, energy extraction).
            </small>
        </div>
    """)
], layout=widgets.Layout(padding='10px'))

# Interactive output
out = widgets.interactive_output(
    update_dashboard, 
    {
        'mass_m': mass_slider, 
        'dist_log': dist_slider,
        'spin': spin_slider,
        'velocity': velocity_slider
    }
)

# Display everything
display(ui, out)

# Initial calculation trigger
mass_slider.value = 100.01
mass_slider.value = 100.0

🌌 RELATIVISTIC SPACETIME ANALYZER 🌌



Output()